In [9]:
import awkward as ak 
import pandas as pd
import numpy as np

In [25]:
def gen_match_data(data, gen_matching_var: str):
    mask = data[gen_matching_var] == 1
    masked_data = {key: ak.mask(data[key], mask) for key in data}
    
    # Drop None values from each array in masked_data
    masked_data = {key: ak.drop_none(value) for key, value in masked_data.items()}
    
    for key in masked_data.keys():
        masked_data[key] = masked_data[key][ak.any(masked_data[key], axis=-1)]
    
    return masked_data

gen_matching_var = "C_pass_gen_matching"

In [26]:
data = {gen_matching_var:
        ak.Array([[0,1],[1],[0,0]]),
        "C_Ds_pt":
        ak.Array([[1.2,3.2],[0.2],[1.,2.]])
}
data

{'C_pass_gen_matching': <Array [[0, 1], [1], [0, 0]] type='3 * var * int64'>,
 'C_Ds_pt': <Array [[1.2, 3.2], [0.2], [1, 2]] type='3 * var * float64'>}

In [27]:
data = gen_match_data(data,gen_matching_var=gen_matching_var)
data

{'C_pass_gen_matching': <Array [[1], [1]] type='2 * var * int64'>,
 'C_Ds_pt': <Array [[3.2], [0.2]] type='2 * var * float64'>}

In [24]:
data = ak.Array([[None, 3.2], [0.2], [None, None]])
filtered_data = ak.drop_none(data)
filtered_data = filtered_data[ak.any(filtered_data, axis=-1)]

print(filtered_data)

[[3.2], [0.2]]
